In [1]:
#usual imports
import numpy as np
import ROOT as r
import matplotlib.pyplot as plt
from root_numpy import tree2array, testdata, list_branches
import pandas as pd
import time
import xgboost as xg
from sklearn.metrics import roc_auc_score, roc_curve
import pickle

trainFrac = 0.7
validFrac = 0.1
nClasses = 9

Welcome to JupyROOT 6.12/06


/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#get trees from files, put them in data frames
procFileMap = {'ggh':'ggH.root', 'dipho':'Dipho.root', 'gjet':'GJet.root', 'qcd':'QCD.root' }
theProcs = procFileMap.keys()

trainDir = '../trainTrees'
trainFrames = {}

for proc,fn in procFileMap.iteritems():
    trainFile   = r.TFile('%s/%s'%(trainDir,fn))
    if 'ggh' in proc or 'vbf' in proc: trainTree = trainFile.Get('vbfTagDumper/trees/%s_125_13TeV_VBFDiJet'%proc)
    else: trainTree = trainFile.Get('vbfTagDumper/trees/%s_13TeV_VBFDiJet'%proc)
    trainFrames[proc] = pd.DataFrame( tree2array(trainTree) )
    trainFrames[proc]['proc'] = proc

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/lib/python2.7/site-packages/root_numpy/_tree.py:385: RuntimeWarning: ignoring duplicate branch named 'nvtx'
  cache_size)


In [3]:
#create one total frame
trainList = []
for proc in theProcs:
    trainList.append(trainFrames[proc])
trainTotal = pd.concat(trainList)

In [4]:
#then filter out the events into only those with the phase space we are interested in
#trainTotal[trainTotal.diphomva>-0.4]
#print 'done dipho mva cut'
trainTotal = trainTotal[trainTotal.CMS_hgg_mass>100.]
print 'done lower mass cut'
trainTotal = trainTotal[trainTotal.CMS_hgg_mass<180.]
print 'done upper mass cut'
trainTotal = trainTotal[trainTotal.stage1cat>-1.]
print 'done first stage 1 cut'
trainTotal = trainTotal[trainTotal.stage1cat<12.]
print 'done second stage 1 cut'
trainTotal = trainTotal[trainTotal.stage1cat!=1]
print 'done third stage 1 cut'
trainTotal = trainTotal[trainTotal.stage1cat!=2]
print 'done fourth and final stage 1 cut'

done lower mass cut
done upper mass cut
done first stage 1 cut
done second stage 1 cut
done third stage 1 cut
done fourth and final stage 1 cut


In [5]:
#add diphoton pt as a column
def addPt(row):
    return row['CMS_hgg_mass']*row['diphoptom']

trainTotal['diphopt'] = trainTotal.apply(addPt, axis=1)

In [13]:
#define the different sets of variables used
diphoVars  = ['leadmva','subleadmva','leadptom','subleadptom',
              'leadeta','subleadeta',
              'CosPhi','vtxprob','sigmarv','sigmawv']
classVars  = ['n_rec_jets','dijet_Mjj','diphopt',
              'dijet_leadEta','dijet_subleadEta','dijet_subsubleadEta',
              #'dijet_LeadJPt','dijet_SubJPt','dijet_SubsubJPt',
              'dijet_LeadJPt','dijet_SubJPt',
              'dijet_leadPUMVA','dijet_subleadPUMVA','dijet_subsubleadPUMVA',
              'dijet_leadDeltaPhi','dijet_subleadDeltaPhi','dijet_subsubleadDeltaPhi',
              'dijet_leadDeltaEta','dijet_subleadDeltaEta','dijet_subsubleadDeltaEta']
trainTotal[diphoVars]

,leadmva,subleadmva,leadptom,subleadptom,leadeta,subleadeta,CosPhi,vtxprob,sigmarv,sigmawv
1,-0.082500,0.977675,0.253160,0.233796,-0.430246,2.274336,-0.940191,0.337356,0.019388,0.021389
5,0.960867,-0.593394,0.380394,0.242821,-2.301987,-0.135242,-0.990953,0.362964,0.027290,0.033188
9,0.972219,0.069912,0.279535,0.255804,-0.145196,2.334992,-0.978791,0.151144,0.046733,0.048833
11,0.383183,0.903200,0.257454,0.182706,-1.252758,1.880495,0.866439,0.907744,0.032817,0.032848
15,-0.542549,0.183245,0.275836,0.257755,0.700013,-1.824525,-0.749920,0.563038,0.024763,0.025208
17,0.383437,0.843853,0.280913,0.168455,-1.332134,1.639903,-0.774641,0.418624,0.025453,0.025458
18,0.890553,0.966199,0.488196,0.238573,1.027406,-0.864465,-0.901668,0.995332,0.009343,0.009396
19,0.960319,-0.807319,0.215122,0.175469,-2.137508,1.072688,-0.833879,0.953720,0.185446,0.185456
21,0.934239,-0.796070,0.275371,0.127151,-1.057373,2.248209,-0.629555,0.990172,0.018649,0.018779
23,0.915768,0.092856,0.373222,0.286160,-1.172423,0.818604,-0.951794,0.652965,0.023384,0.023465


In [7]:
#add column corresponding to truth bin
def truthDipho(row):
    if not row['stage1cat']==0: return 1
    else: return 0

trainTotal['truthDipho'] = trainTotal.apply(truthDipho,axis=1)

In [8]:
#add column corresponding to truth bin
def truthClass(row):
    if not row['stage1cat']==0: return int(row['stage1cat']-3)
    else: return 0

trainTotal['truthClass'] = trainTotal.apply(truthClass,axis=1)

In [9]:
#add column corresponding to reco bin
def reco(row):
    if row['n_rec_jets']==0: return 0
    elif row['n_rec_jets']==1:
        if row['diphopt'] < 60: return 1
        elif row['diphopt'] < 120: return 2
        elif row['diphopt'] < 200: return 3
        else: return 4
    else:
        if row['diphopt'] < 60: return 5
        elif row['diphopt'] < 120: return 6
        elif row['diphopt'] < 200: return 7
        else: return 8


trainTotal['reco'] = trainTotal.apply(reco,axis=1)

In [10]:
#add column with full weights for testing
def fullWeight(row):
    if row['proc'] == 'dipho': return 3. * row['weight'] #only using 1/3 of the diphoton sample
    else: return row['weight']
trainTotal['fullWeight'] = trainTotal.apply(fullWeight,axis=1)

In [11]:
#add column with *normalised weights for training only*
weightFactors = [1., 0.0002994, 0.0000757, 0.0000530, 0.0000099, 0.0000029, 0.0000154, 0.0000235, 0.0000165, 0.0000104]
def normWeight(row):
    weight = row['weight']
    if row['proc'] == 'dipho': 
        weight *= 3. / weightFactors[ int(row['truthClass']) ] #only using 1/3 of the diphoton sample
    elif row['proc'] == 'qcd': 
        weight *= 0.04 / weightFactors[ int(row['truthClass']) ] #reduce because too large by default
    else: 
        weight *= 1. / weightFactors[ int(row['truthClass']) ] #otherwise just reweight by xs
    #now account for the resolution
    if row['sigmarv']>0. and row['sigmawv']>0.:
        weight *= ( (row['vtxprob']/row['sigmarv']) + ((1.-row['vtxprob'])/row['sigmawv']) )
    weight = abs(weight)
    return weight
trainTotal['normedWeight'] = trainTotal.apply(normWeight,axis=1)

#elif row['truth'] == 1: return 5. * row['weight'] / weightFactors[ int(row['truth']) ] #optional ad-hoc factor
#trainTotal['normedWeight']

In [18]:
#all necessary variables added - good idea to save at this point! 
trainTotal.to_pickle('trainTotal.pkl')

In [14]:
#setup the various datasets
#FIXME need to fix this to *drop* the background from the multi-class training
allC = trainTotal[classVars].values #multi-classifier input variables
allD = trainTotal[diphoVars].values #diphoton BDT input variables
allY = trainTotal['truthClass'].values #signal or background
allZ = trainTotal['truthDipho'].values #truth/target values (i.e. the gen-level Stage 1 bins)
allFW = trainTotal['fullWeight'].values #weights corresponding to number of events
allNW = trainTotal['normedWeight'].values #normalised weights for training - can be played with
allR = trainTotal['reco'].values
allM = trainTotal['CMS_hgg_mass'].values
shuffle = np.random.permutation(allC.shape[0])
allC = allC[shuffle]
allD = allD[shuffle]
allY = allY[shuffle]
allZ = allZ[shuffle]
allFW = allFW[shuffle]
allNW = allNW[shuffle]
allR = allR[shuffle]
allM = allM[shuffle]
trainLimit = int(allC.shape[0]*trainFrac)
validLimit = int(allC.shape[0]*(trainFrac+validFrac))
trainC, validC, testC = np.split( allC, [trainLimit,validLimit] )
trainD, validD, testD = np.split( allD, [trainLimit,validLimit] )
trainY, validY, testY = np.split( allY, [trainLimit,validLimit] )
trainZ, validZ, testZ = np.split( allZ, [trainLimit,validLimit] )
trainFW, validFW, testFW = np.split( allFW, [trainLimit,validLimit] )
trainNW, validNW, testNW = np.split( allNW, [trainLimit,validLimit] )
trainR, validR, testR = np.split( allR, [trainLimit,validLimit] )
trainM, validM, testM = np.split( allM, [trainLimit,validLimit] )

In [15]:
#build the multi-classifier
trainingClass = xg.DMatrix(trainC, label=trainY, weight=trainNW, feature_names=classVars)
testingClass  = xg.DMatrix(testC, label=testY, weight=testFW, feature_names=classVars)
paramClass = {}
paramClass['objective'] = 'multi:softprob'
paramClass['num_class'] = nClasses
classModel = xg.train(paramClass, trainingClass)

#save it
classModel.save_model('classModel.model')
pickle.dump(classModel, open("classModel.pickle.dat", "wb"))

[18:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[18:48:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[18:48:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[18:48:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[18:48:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[18:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[18:48:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[18:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[18:49:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nod

In [17]:
#build the background discrimination BDT
trainingDipho = xg.DMatrix(trainD, label=trainZ, weight=trainNW, feature_names=diphoVars)
testingDipho  = xg.DMatrix(testD, label=testZ, weight=testFW, feature_names=diphoVars)
paramDipho = {}
paramDipho['objective'] = 'binary:logistic'
diphoModel = xg.train(paramDipho, trainingDipho)

#save it
diphoModel.save_model('diphoModel.model')
pickle.dump(diphoModel, open("diphoModel.pickle.dat", "wb"))

[19:09:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[19:09:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[19:10:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[19:10:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[19:10:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[19:10:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[19:10:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[19:10:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[19:11:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned no

In [34]:
#get predicted values
predProbClass = classModel.predict(testingClass).reshape(testY.shape[0],nClasses) #FIXME: not sure if this is right
predY = np.argmax(predProbClass, axis=1)
print testY
print predY
print

predProbDipho = diphoModel.predict(testingDipho)
print testZ
print predProbDipho
print roc_auc_score(testZ, predProbDipho, sample_weight=testFW)
roc_curve(testZ, predProbDipho, sample_weight=testFW)

[0 2 2 ..., 0 0 0]
[5 2 6 ..., 5 2 7]

[0 1 1 ..., 0 0 0]
[ 0.2160857   0.92750895  0.97573304 ...,  0.49987105  0.96475571
  0.9716385 ]
0.814326669778


(array([ 0.        ,  0.00160566,  0.00161531, ...,  0.99995486,
         0.9999552 ,  1.        ]),
 array([ 0.        ,  0.02913263,  0.02978483, ...,  1.        ,
         1.        ,  1.        ]),
 array([  1.97736502e+00,   9.77364957e-01,   9.77251410e-01, ...,
          5.88232477e-04,   5.80993074e-04,   3.18984268e-04], dtype=float32))

In [ ]:
#setup 2D hists
truthHist = r.TH1F('truthHist','truthHist',nClasses,-0.5,nClasses-0.5)
predHist  = r.TH1F('predHist','predHist',nClasses,-0.5,nClasses-0.5)
rightHist = r.TH1F('rightHist','rightHist',nClasses,-0.5,nClasses-0.5)
wrongHist = r.TH1F('wrongHist','wrongHist',nClasses,-0.5,nClasses-0.5)
for true,guess,w in zip(testY,pred,testFW):
    truthHist.Fill(true,w)
    predHist.Fill(guess,w)
    if true==guess: rightHist.Fill(true,w)
    else: wrongHist.Fill(true,w)
firstBinVal = -1.
for iBin in range(1,truthHist.GetNbinsX()+1):
    if iBin==1: firstBinVal = truthHist.GetBinContent(iBin)
    ratio = float(truthHist.GetBinContent(iBin)) / firstBinVal
    print 'ratio for bin %g is %1.7f'%(iBin,ratio)
wrongHist.Add(rightHist)
rightHist.Divide(wrongHist)
effHist = r.TH1F
r.gStyle.SetOptStat(0)
canv = r.TCanvas()
canv.cd()
truthHist.GetYaxis().SetRangeUser(0.,6.)
truthHist.Draw('hist')
canv.Print('truthHist.pdf')
predHist.GetYaxis().SetRangeUser(0.,6.)
predHist.Draw('hist')
canv.Print('predHist.pdf')
rightHist.GetYaxis().SetRangeUser(0.,1.)
rightHist.Draw('hist')
canv.Print('efficiencyHist.pdf')

In [ ]:
#generate weights for the 2D hists    
sumwProcCatMapReco = {}
sumwProcCatMapPred = {}
sumwProcMap = {}
for iProc in range(1, nClasses):
    sumwProcMap[iProc] = np.sum(testFW*(testY==iProc))
    for jProc in range(nClasses):
        sumwProcCatMapPred[(iProc,jProc)] = np.sum(testFW*(testY==iProc)*(pred==jProc))
        sumwProcCatMapReco[(iProc,jProc)] = np.sum(testFW*(testY==iProc)*(testR==jProc))
sumwCatMapReco = {}
sumwCatMapPred = {}
for iProc in range(1, nClasses):
    #sumwCatMapPred[iProc] = np.sum(testFW*(pred==iProc))
    #sumwCatMapReco[iProc] = np.sum(testFW*(testR==iProc)
    sumwCatMapPred[iProc] = np.sum(testFW*(pred==iProc)*(testY!=0)) #don't count bkg here
    sumwCatMapReco[iProc] = np.sum(testFW*(testR==iProc)*(testY!=0))

In [ ]:
#fill the 2D hists
nBinsX=nClasses
nBinsY=nClasses
procHistReco = r.TH2F('procHistReco','procHistReco', nBinsX, -0.5, nBinsX-0.5, nBinsY, -0.5, nBinsY-0.5)
procHistReco.SetTitle('')
procHistPred = r.TH2F('procHistPred','procHistPred', nBinsX, -0.5, nBinsX-0.5, nBinsY, -0.5, nBinsY-0.5)
procHistPred.SetTitle('')
catHistReco  = r.TH2F('catHistReco','catHistReco', nBinsX, -0.5, nBinsX-0.5, nBinsY, -0.5, nBinsY-0.5)
catHistReco.SetTitle('')
catHistPred  = r.TH2F('catHistPred','catHistPred', nBinsX, -0.5, nBinsX-0.5, nBinsY, -0.5, nBinsY-0.5)
catHistPred.SetTitle('')
for iProc in range(1, nClasses):
    for jProc in range(1, nClasses):
        procWeightReco = 100. * sumwProcCatMapReco[(iProc,jProc)] / sumwProcMap[iProc]
        procWeightPred = 100. * sumwProcCatMapPred[(iProc,jProc)] / sumwProcMap[iProc]
        catWeightReco  = 100. * sumwProcCatMapReco[(iProc,jProc)] / sumwCatMapReco[jProc]
        catWeightPred  = 100. * sumwProcCatMapPred[(iProc,jProc)] / sumwCatMapPred[jProc]
        
        procHistReco.Fill(iProc, jProc, procWeightReco)
        procHistPred.Fill(iProc, jProc, procWeightPred)
        catHistReco.Fill(iProc, jProc, catWeightReco)
        catHistPred.Fill(iProc, jProc, catWeightPred)

In [ ]:
#draw the 2D hists
def prettyHist(hist):
    hist.SetStats(0)
    hist.GetXaxis().SetTitle('Process')
    hist.GetXaxis().SetTickLength(0.)
    hist.GetYaxis().SetTitle('Category')
    hist.GetYaxis().SetTitleOffset(1.5)
    hist.GetYaxis().SetTickLength(0.)
    hist.SetMinimum(-0.00001)
    hist.SetMaximum(100.)
    
canv = r.TCanvas()
r.gStyle.SetPaintTextFormat('2.0f')
prettyHist(procHistReco)
procHistReco.Draw('colz,text')
canv.Print('procHistReco.pdf')
prettyHist(catHistReco)
catHistReco.Draw('colz,text')
canv.Print('catHistReco.pdf')
prettyHist(procHistPred)
procHistPred.Draw('colz,text')
canv.Print('procHistPred.pdf')
prettyHist(catHistPred)
catHistPred.Draw('colz,text')
canv.Print('catHistPred.pdf')

In [ ]:
#draw the bkg-prob distribution for each category
bkgHists = {}
sigHists = {}
bkgProb = predProb[:,0]*(testY==0)
for iProc in range(1, nClasses):
    sigProb = predProb[:,0]*(testY==iProc) #could do this for inclusive signal? Or add other signal to background...
    bkgHists[iProc] = r.TH1F('bkgProb_%s'%iProc, 'bkgProb_%s'%iProc, 50, 0., 1.)
    sigHists[iProc] = r.TH1F('sigProb_%s'%iProc, 'sigProb_%s'%iProc, 50, 0., 1.)
    for bkg,sig,w in zip(bkgProb,sigProb,testFW):
        if bkg!=0: bkgHists[iProc].Fill(1.-bkg,w)
        if sig!=0: sigHists[iProc].Fill(1.-sig,w)
canv = r.TCanvas()
for iProc in range(1, nClasses):
    bkgInt = bkgHists[iProc].Integral()
    bkgHists[iProc].Scale(1./bkgInt)
    bkgHists[iProc].SetLineColor(r.kBlack)
    sigInt = sigHists[iProc].Integral()
    sigHists[iProc].Scale(1./sigInt)
    sigHists[iProc].SetLineColor(r.kGreen+1)
    sigHists[iProc].Draw('hist')
    bkgHists[iProc].Draw('hist,same')
    canv.Print('bkgProb_%s.pdf'%iProc)

In [ ]:
#calculate (very) naive significance
lumi = 77.
for iProc in range(1, nClasses):
    sigReco = lumi * sumwProcCatMapReco[(iProc,iProc)]
    sigPred = lumi * sumwProcCatMapPred[(iProc,iProc)]
    totReco = lumi * sumwCatMapReco[iProc]
    totPred = lumi * sumwCatMapPred[iProc]
    bkgReco = lumi * np.sum(testFW * (testY==0) * (pred==iProc) * (testM>123.) * (testM<127.) * (bkgProb<0.08))
    bkgPred = lumi * np.sum(testFW * (testY==0) * (pred==iProc) * (testM>123.) * (testM<127.) * (bkgProb<0.08))
    valReco = sigReco / np.sqrt(totReco+bkgReco)
    valPred = sigPred / np.sqrt(totPred+bkgPred)
    print 'for class %g the S and B counts are:'%iProc
    print 'Reco: %1.3f and %1.3f,   BDT %1.3f and %1.3f'%(sigReco, totReco+bkgReco, sigPred, totPred+bkgPred)
    print 'corresponding to naive significance of:'
    print 'Reco: %1.3f,   BDT %1.3f \n'%(valReco, valPred)

In [ ]:
# get feature importances
xg.plot_importance(classifier)
plt.show()

In [ ]:
#define rms function
def getWidth(array):
    sumW = np.sum(array)
    mean = np.sum(array*testM)/ sumW
    diff = testM - mean
    diffSq = diff*diff
    sumDiffSq = np.sum(diffSq*array) / sumW
    width = np.sqrt(sumDiffSq)
    return width

In [ ]:
#define approximate mean significance
def getAMS(s, b, breg=1.):
    b = b + breg
    val = 0.
    if b > 0.:
        val = (s + b)*np.log(1. + (s/b))
        val = 2*(val - s)
        val = np.sqrt(val)
    return val

In [ ]:
#going to try two different significance estimates - hopefully they agree well.. 
#first is to use only numpy, i.e. no ROOT, so a bit cruder bc no bkg fit and rms instead of effSigma
lumi = 77.
theProbs = 1. - predProb[:,0]

#first do one category
for iProc in range(1, nClasses):
    bestCut = -1.
    bestSignif = -1.
    bestSig = -1.
    bestBkg = -1.
    #for cut in np.random.rand(100,1):
    for cut in np.arange(0.9,1.0,0.0005):
        sigArray = testFW * (testY==iProc) * (pred==iProc) * (theProbs>cut)
        sigPred = lumi * 0.68 * np.sum( sigArray )
        #sigRMS = getWidth(sigArray)
        sigRMS = 0.67 * getWidth(sigArray) #ad hoc because rms seems to be an overestimate
        #if cut < 0.9501 and cut > 0.9495: print 'sig RMS in proc %g is %1.3f'%(iProc,sigRMS)
        totArray = testFW * (testY!=0) * (pred==iProc) * (theProbs>cut)
        totPred = lumi * 0.68 * np.sum( totArray )
        bkgArray = testFW * (testY==0) * (pred==iProc) * (testM>(125.-sigRMS)) * (testM<(125.+sigRMS)) * (theProbs>cut)
        bkgPred = lumi * np.sum(bkgArray)
        fullBkg = bkgPred + totPred - sigPred
        valPred = getAMS(sigPred, fullBkg)
        #print 'cut is %1.3f'%cut
        #print 'rms is %1.2f'%sigRMS
        #print 'S, B are %1.2f, %1.2f'%(sigPred,fullBkg)
        #print 'signif is %1.2f'%valPred
        if valPred > bestSignif: 
            bestCut = cut
            bestSignif = valPred
            bestSig = sigPred
            bestBkg = fullBkg
    print 'optimal one-category signifiance for proc %s:'%(iProc)
    print 'cut = %1.3f, S = %1.2f, B = %1.2f, signif = %1.2f'%(bestCut, bestSig, bestBkg, bestSignif)
    print

In [ ]:
#now try two categories
for iProc in range(1, nClasses):
    bestLoCut = -1.
    bestHiCut = -1.
    bestSignif = -1.
    for cut1 in np.random.rand(50,1):
        for cut2 in np.random.rand(50,1):
            loCut = min(cut1, cut2)
            hiCut = max(cut1, cut2)
            #get significance of first (purest) cat
            sigArray = testFW * (testY==iProc) * (pred==iProc) * (theProbs>hiCut)
            sigPred = lumi * 0.68 * np.sum( sigArray )
            sigRMS = getWidth(sigArray)
            totArray = testFW * (testY!=0) * (pred==iProc) * (theProbs>hiCut)
            totPred = lumi * 0.68 * np.sum( totArray )
            bkgArray = testFW * (testY==0) * (pred==iProc) * (testM>(125.-sigRMS)) * (testM<(125.+sigRMS)) * (theProbs>hiCut)
            bkgPred = lumi * np.sum(bkgArray)
            fullBkg = bkgPred + totPred - sigPred
            valPred_0 = getAMS(sigPred, fullBkg)
            #get significance of second (purest) cat
            sigArray = testFW * (testY==iProc) * (pred==iProc) * (theProbs>loCut) * (theProbs<hiCut)
            sigPred = lumi * 0.68 * np.sum( sigArray )
            sigRMS = getWidth(sigArray)
            totArray = testFW * (testY!=0) * (pred==iProc) * (theProbs>loCut) * (theProbs<hiCut)
            totPred = lumi * 0.68 * np.sum( totArray )
            bkgArray = testFW * (testY==0) * (pred==iProc) * (testM>(125.-sigRMS)) * (testM<(125.+sigRMS)) * (theProbs>loCut) * (theProbs<hiCut)
            bkgPred = lumi * np.sum(bkgArray)
            fullBkg = bkgPred + totPred - sigPred
            valPred_1 = np.getAMS(sigPred, fullBkg)
            valPred = np.sqrt( valPred_0*valPred_0 + valPred_1*valPred_1)
            if valPred > bestSignif: 
                bestLoCut = loCut
                bestHiCut = hiCut
                bestSignif = valPred
    print 'optimal two-category signifiance for proc %s:'%(iProc)
    print 'cutLow = %1.2f, cutHigh = %1.2f, signif = %1.2f'%(bestLoCut, bestHiCut, bestSignif)
    print

In [ ]:
#now move on to the more "traditional method", which uses a fit to bkg distribution and full sigmaEff calc
#set up 2D hists in diphoMass, bkgProb
diagSigHists = {}
fullSigHists = {}
bkgOnlyHists = {}
nBins = 100
for iProc in range(1, nClasses):
    diagSigHists[iProc] = r.TH2F('diagSigHist_%g'%iProc, 'diagSigHist_%g'%iProc, 10*nBins, 0., 1., nBins, 100., 180.)
    fullSigHists[iProc] = r.TH2F('fullSigHist_%g'%iProc, 'fullSigHist_%g'%iProc, 10*nBins, 0., 1., nBins, 100., 180.)
    bkgOnlyHists[iProc] = r.TH2F('bkgOnlyHist_%g'%iProc, 'bkgOnlyHist_%g'%iProc, 10*nBins, 0., 1., nBins, 100., 180.)
    diagSigW = testFW * (testY==iProc) * (pred==iProc)
    fullSigW = testFW * (testY!=0) * (pred==iProc)
    bkgOnlyW = testFW * (testY==0) * (pred==iProc)
    for mass,prob,diagW,fullW,bkgW in zip(testM,theProbs,diagSigW,fullSigW,bkgOnlyW):
        diagSigHists[iProc].Fill(prob, mass, diagW)
        fullSigHists[iProc].Fill(prob, mass, fullW)
        bkgOnlyHists[iProc].Fill(prob, mass, bkgW)

In [ ]:
#one-category optimisation
from diphoHelpers import evalSignif
for iProc in range(1, nClasses):
    bestCut = -1.
    bestSignif = -1.
    for cut in np.arange(0.9,1.0,0.001):
        #print 'checking cut of %1.3f'%cut
        signif = evalSignif( diagSigHists, fullSigHists, bkgOnlyHists, iProc, int(10*nBins*cut), 10*nBins, True )
        #print 'its signif is %1.3f'%signif
        if signif > bestSignif:
            bestCut = cut
            bestSignif = signif
    print 'optimal one-category signifiance for proc %s:'%(iProc)
    print 'cut = %1.3f, signif = %1.2f'%(bestCut, bestSignif)
    print

In [ ]:
#check the background distributions look sensible
checkHist = bkgOnlyHists[1].ProjectionY('tempProj', 900, 1000)
#checkHist = bkgOnlyHists[1]
canv = r.TCanvas()
checkHist.Draw('hist')
#checkHist.Draw('colz')
canv.Print('checkHist.pdf')